#### INSTALLATIONS

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.4 MB/s eta 0:00:00


In [ ]:
pip install --upgrade scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### IMPORTING IMBD DATASET

In [ ]:
from datasets import load_dataset
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

####PREPROCESSING

In [ ]:
import torch
import pandas as pd
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW
from sklearn.metrics import accuracy_score

# Loading the DistilBERT tokenizer and encode the data
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
train_encoded = train_dataset.map(tokenize, batched=True)
test_encoded = test_dataset.map(tokenize, batched=True)

# Convert the encoded dataset to a PyTorch DataLoader
train_loader = torch.utils.data.DataLoader(train_encoded, batch_size=16, shuffle=True,
                                           collate_fn=lambda batch: {'input_ids': torch.tensor([b['input_ids'] for b in batch], dtype=torch.long),
                                                                    'attention_mask': torch.tensor([b['attention_mask'] for b in batch], dtype=torch.long),
                                                                    'label': torch.tensor([b['label'] for b in batch], dtype=torch.long)})
test_loader = torch.utils.data.DataLoader(test_encoded, batch_size=16, shuffle=True,
                                           collate_fn=lambda batch: {'input_ids': torch.tensor([b['input_ids'] for b in batch], dtype=torch.long),
                                                                    'attention_mask': torch.tensor([b['attention_mask'] for b in batch], dtype=torch.long),
                                                                    'label': torch.tensor([b['label'] for b in batch], dtype=torch.long)})




Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
#function to train the data
def train(model):
  from tqdm import tqdm
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  model.to(device)
  model.train()#set model to training mode

  # Train the model for 3 epochs
  for epoch in range(3):
      running_loss = 0.0
      progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")#show the progress of the epochs
      for batch in progress_bar:
          input_ids = torch.tensor(batch['input_ids']).to(device)#converting the input batches to tensors
          attention_mask = torch.tensor(batch['attention_mask']).to(device)
          labels = torch.tensor(batch['label']).to(device)#creating labels tensor
          optimizer.zero_grad()#setting the optimizer parameters to zero
          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)#storing outputs from model
          loss = outputs.loss#computing the loss function
          loss.backward()#back propagation
          optimizer.step()#updating parameters
          running_loss += loss.item()
          progress_bar.set_postfix({"Loss value": f"{running_loss / len(train_loader):.5f}"})#display loss for each epoch




In [ ]:
#fuction to Evaluate the performance using test dataset
from tqdm import tqdm
def test_data_eval(model):
  model.eval()
  y_actual = []
  y_predictions = []
  with torch.no_grad():
    progress_bar = tqdm(test_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
      input_ids = torch.tensor(batch['input_ids']).to(device)
      attention_mask = torch.tensor(batch['attention_mask']).to(device)
      labels = torch.tensor(batch['label']).to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=1).detach().cpu().tolist()
      y_predictions += predictions
      y_actual += labels.detach().cpu().tolist()
  return y_actual,y_predictions


####FINE TUNING

In [ ]:
#loading the pretrained model distilbert
model_fine_tuning = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')# Load the pre-trained model 
model_fine_tuning.classifier = torch.nn.Linear(in_features=768, out_features=2, bias=True) # Adding linear classifier 
loss_function = torch.nn.CrossEntropyLoss()# adamW optimizer and cross entropy loss function
optimizer = AdamW(model_fine_tuning.parameters(), lr=1e-5)#learning rate as 1e-5
train(model_fine_tuning)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [ ]:
a,b = test_data_eval(model_fine_tuning)
accuracy_model_1 = accuracy_score(a,b)
print("\n Accuracy:",accuracy_model_1)

NameError: ignored

#### LINEAR PROBING

In [ ]:
# freezing all layers except the classifier
model_linear_probing = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
for param in model_linear_probing.parameters():
    param.requires_grad = False
for param in model_linear_probing.classifier.parameters():
    param.requires_grad = True


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_linear_probing.to(device)
model_linear_probing.train()#set model to training mode
train(model_linear_probing)

Epoch 1:   0%|          | 0/1563 [00:00<?, ?it/s]<ipython-input-24-4b508a3b87d3>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-24-4b508a3b87d3>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-24-4b508a3b87d3>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)
Epoch 3: 100%|██████████| 1563/1563 [07:26<00:00,  3.50it/s, Loss value=0.6970

In [ ]:
a,b = test_data_eval(model_linear_probing)
accuracy_model_2= accuracy_score(a,b)
print('\n Accuracy:', accuracy_model_2)

Epoch 3:   0%|          | 0/1563 [00:00<?, ?it/s]<ipython-input-11-b856ab95fa38>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-11-b856ab95fa38>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-11-b856ab95fa38>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)
Epoch 3: 100%|██████████| 1563/1563 [06:49<00:00,  3.81it/s]

Accuracy: 0.49976


#### BASE MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import gensim.downloader as api
from sklearn.metrics import accuracy_score


word_vectors = api.load("glove-twitter-25")# download word embeddings

#model architecture
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, embedding_dim, weights):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding.from_pretrained(weights)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = hidden[-1]
        out = self.fc(hidden)
        return out

#hyperparameters
batch_size = 50
learning_rate = 1e-3
num_epochs = 3

# Initialize the model
vocab_size = len(word_vectors.key_to_index)
embedding_dim = word_vectors.vector_size
hidden_dim = 128
output_dim = 2
weights = torch.from_numpy(np.array([word_vectors[word] for word in word_vectors.key_to_index]))
model = LSTMClassifier(vocab_size, hidden_dim, output_dim, embedding_dim, weights).to(device)

# loss function and optimizer
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



[=================================================-] 98.7% 103.4/104.8MB downloaded

In [ ]:

# Train the model
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        text = batch['input_ids'].to(device)
        label = batch['label'].to(device)
        output = model(text)
        loss = loss(output, label)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")

# Evaluate the model on the test set
model.eval()
y_actual = []
y_predictions = []
with torch.no_grad():
    for batch in test_loader:
        text = batch['input_ids'].to(device)
        label = batch['label'].to(device)
        output = model(text)
        predictions = torch.argmax(output, dim=1).detach().cpu().tolist()
        y_predictions += predictions
        y_actual += label.detach().cpu().tolist()

Epoch 1 Loss: 0.6838409900665283
Epoch 2 Loss: 0.697978138923645
Epoch 3 Loss: 0.6711217761039734


In [ ]:
accuracy_base_model = accuracy_score(y_actual, y_predictions)
print('\n Accuracy:', accuracy_base_model)


Accuracy: 0.50524


####COMPARISION

In [ ]:
print('accuracy of the base model is:',accuracy_base_model)
print('accuracy of fine tuning model is:',accuracy_model_1)
print('accuracy of linear probed model is:',accuracy_model_2)

accuracy of the base model is: 0.50524
accuracy of fine tuning model is: 0.93172
accuracy of linear probed model is: 0.49976
